### Nome: **Rodrigo Meneses Rodarte**
### Ano: **2022**
### Trabalho de Conclusão de Curso: **Recomendação de produtos baseada em análise de sentimentos**





In [ ]:
!pip install transformers seqeval[gpu]
!pip install evaluate
!pip install bert_score


In [40]:
import pandas as pd
import numpy as np

import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from evaluate import load
from termcolor import colored
from seqeval.metrics import classification_report

from transformers import AutoModel, BertTokenizerFast, BertConfig, BertForTokenClassification, AutoTokenizer, AutoModelForPreTraining


MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 7
LEARNING_RATE = 2e-05
MAX_GRAD_NORM = 10 

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=True)
bertscore = load("bertscore")

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)
df = pd.read_csv ('/content/sample_data/B2W-Reviews01.csv')
df.shape

In [ ]:
##Removing unecessary columns of original B2W Data Frame
df = df.drop(['submission_date', 'reviewer_id', 'product_id', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state'],axis=1)
df.shape

In [ ]:
## Reading the User Input
name = (input("Digite o produto que deseja pesquisar: "))
print ("Produto selecionado pelo usuário:", name)

new_df = df.loc[df['product_name'].str.contains(name, na= False, case=False)]
print ("Data Frame with User Product", new_df.shape)
print(new_df['site_category_lv2'].unique())
print(new_df['site_category_lv1'].unique())


### Ordering the product by the overall rating
new_df_sorted = new_df.sort_values(by=['overall_rating'], ascending=False)
new_df_sorted.to_excel("filtered.xlsx")  


print(new_df_sorted.shape)
print((new_df_sorted['overall_rating']).value_counts())

review_text_np_array = new_df_sorted['review_text'].to_numpy()

In [ ]:
# Appending out IOB tagged DATA
df_final = pd.DataFrame() # Data Frame that will be used for fine tunning BERT with tagged sentences

data_smartphone = pd.read_csv ('/content/sample_data/domain_smartphone_IOB.csv', on_bad_lines='skip', sep='\t')
df_final = df_final.append(data_smartphone)

data_camera = pd.read_csv ('/content//sample_data/domain_camera_IOB.csv', on_bad_lines='skip', sep='\t')
df_final = df_final.append(data_camera)

data_hotel = pd.read_csv ('/content//sample_data/domain_hotel_IOB.csv', on_bad_lines='skip', sep='\t')
df_final = df_final.append(data_hotel)

data_livro = pd.read_csv ('/content//sample_data/domain_livro_IOB.csv', on_bad_lines='skip', sep='\t')
df_final = df_final.append(data_livro)

df_final.count()

print("Number of tags: {}".format(len(df_final.Tag.unique())))
frequencies = df_final.Tag.value_counts()
frequencies

df_final = df_final.fillna(method='ffill')
#df_final.head(50)

# let's create a new column called "sentence" which groups the words by sentence 
df_final['sentence'] = df_final[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
df_final['word_labels'] = df_final[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
df_final.head()


label2id = {k: v for v, k in enumerate(df_final.Tag.unique())}
id2label = {v: k for v, k in enumerate(df_final.Tag.unique())}
label2id

df_final = df_final[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
df_final

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

train_size = 0.8

## DataSet for Training and testing
train_dataset = df_final.sample(frac=train_size,random_state=200)
test_dataset = df_final.drop(train_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_final.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)
training_set[0]
training_set[0]["ids"]

# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


##Model for training for Aspect Extraction
model = BertForTokenClassification.from_pretrained('neuralmind/bert-large-portuguese-cased', num_labels=len(id2label), id2label=id2label, label2id=label2id)
model.to(device)

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss


tr_logits = outputs[1]
tr_logits.shape

##Optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [48]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.6516180038452148
Training loss per 100 training steps: 0.1582063850162938
Training loss epoch: 0.12307476019486785
Training accuracy epoch: 0.9017291631680863
Training epoch: 2
Training loss per 100 training steps: 0.11329752951860428
Training loss per 100 training steps: 0.06958519877626164
Training loss epoch: 0.07022548134237067
Training accuracy epoch: 0.9349119588344483
Training epoch: 3
Training loss per 100 training steps: 0.0024560990277677774
Training loss per 100 training steps: 0.05247022931943509
Training loss epoch: 0.052318988226896596
Training accuracy epoch: 0.9506598533791242
Training epoch: 4
Training loss per 100 training steps: 0.03411787375807762
Training loss per 100 training steps: 0.04001493204043865
Training loss epoch: 0.038985901739035454
Training accuracy epoch: 0.9635932266832692
Training epoch: 5
Training loss per 100 training steps: 0.029925202950835228
Training loss per 100 training steps: 0.02748

In [49]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [50]:
labels, predictions = valid(model, testing_loader)
print(classification_report([labels], [predictions]))

Validation loss per 100 evaluation steps: 0.10894729197025299
Validation Loss: 0.12482348607843552
Validation Accuracy: 0.9265186961070195
              precision    recall  f1-score   support

         asp       0.67      0.56      0.61       680

   micro avg       0.67      0.56      0.61       680
   macro avg       0.67      0.56      0.61       680
weighted avg       0.67      0.56      0.61       680



In [51]:
def predict_review_text (sentence):

  inputs = tokenizer(sentence, padding='max_length', add_special_tokens=True,  truncation=True, max_length=MAX_LEN, return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, mask)
  logits = outputs[0]
  
  # list of aspects 
  list_of_aspects = []

  outputs.logits

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
  
  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  
  token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
  
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)
 
  #print ("Rodarte wp_preds" , wp_preds)
  word_level_predictions = []

  for pair in wp_preds:
    if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']) :
      # skip prediction
      continue
    elif ((pair[1] in ['B-asp'])):
      #print("Rodartinho", pair[0])
      list_of_aspects.append(pair[0])
      word_level_predictions.append(pair[1])
    else:
      word_level_predictions.append(pair[1])


  # we join tokens, if they are not special ones
  str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
  
  #print(str_rep)
  #print(word_level_predictions)
  
  return (list_of_aspects)

In [ ]:
review_test = []
review_test = new_df_sorted['review_text']
review_test = review_test.values.tolist()


##List of aspects
list_of_aspects = []

for review in review_test:
  print("Rodarte: review", review)
  if ((type(review) is float)):
    continue
  
  aspect = predict_review_text(review)
  list_of_aspects.append(aspect)
print("List of aspects", list_of_aspects)

len(list_of_aspects)


output = []
for x in list_of_aspects:
    if x not in output:
        output.append(x)
print(output)

len(output)

mystring = []
def recursiveLookThrough(arr):
    #mystring = []
    for element in arr:
        if (isinstance(element,list)):
            recursiveLookThrough(element)
        else:
            if element not in mystring:
              mystring.append(element)

    return(mystring)   

string_of_aspects = recursiveLookThrough(output)

count = 0
while count < len(string_of_aspects):
    if ((string_of_aspects[count].startswith("##"))):
      string_of_aspects[count] = string_of_aspects[count].replace('##','')
      string_of_aspects[count-1] = ''.join(string_of_aspects[count-1] + string_of_aspects[count])

      del string_of_aspects[count]
      count = count - 1

    count = count + 1

len(string_of_aspects)


# Removing duplicated itens
final_string_of_aspects = []

In [ ]:
list_of_aspects

In [ ]:
for x in string_of_aspects:
    if x not in final_string_of_aspects:
        final_string_of_aspects.append(x)
print(final_string_of_aspects)

len(final_string_of_aspects)


171

In [ ]:
## list of camera for TCC
##list_of_aspect_selected_by_user = ['preço','preco','custo','desempenho','rapidez',
##                                   'bateria','qualidade','eficaz','camera',
##                                   'design', 'filmadora', 'apple', 'video','sincroniza',
 ##                                  'capacidade', 'robusto','fotosao']

#Celular
#list_of_aspect_selected_by_user = ['celular','sinal','bateria','dura',
#                                   'telefone','bonito','eficaz','qualidade','software',
#                                   'moderno','leve','tecnologia','util',
#                                   'resistente','rapido']

#TV
#list_of_aspect_selected_by_user = ['navegacao','internet','imagem','trava',
#                                   'som','desempenho','graves','tecnologia','nitida',
#                                   'qualidade','interatividade','design','leve',
#                                   'wifi','contraste','cores','interface' ,'acabamento' ,'resolucao' 
#                                   ,'tela','brilho','lenta','performance','sinal']

#Geladeira
list_of_aspect_selected_by_user = ['silenciosa','barulho','volume','design',
                                   'tecnologia','custo',
                                   'desempenho','consumo','custoo','funcionamento',
                                   'som','performance', 'preco','preço','compartimento',
                                   'qualidade','moderno','iluminação','touch']



#livro
#list_of_aspect_selected_by_user = ['divertido','ilustrações','linguagem','literatura','aventura',
#                                   'trama','infantil','poesia','didático','design']


print (len(list_of_aspect_selected_by_user))

for x in list_of_aspect_selected_by_user:
  new_df_sorted.loc[new_df_sorted['review_text'].str.contains(x, na= False, case=False) == True , x ] = 1
  new_df_sorted.loc[new_df_sorted['review_text'].str.contains(x, na= False, case=False) == False , x ] = 0

In [ ]:
temp_df = new_df_sorted[list_of_aspect_selected_by_user]

In [ ]:
new_df_sorted['rank'] = 0
new_df_sorted ['Total_Sum_Aspects'] = temp_df.sum(axis=1)

new_df_sorted.sort_values(by=['Total_Sum_Aspects'], ascending=False)

new_df_sorted['rank'] = new_df_sorted['Total_Sum_Aspects'] * new_df_sorted['overall_rating'] / len(list_of_aspect_selected_by_user)

new_df_sorted = new_df_sorted.sort_values(by=['rank'], ascending=False)
new_df_sorted

In [ ]:
new_df_sorted_product_recommendation = new_df_sorted[['product_name','review_title','review_text','Total_Sum_Aspects','rank','overall_rating']]
new_df_sorted_product_recommendation['count'] = 0
new_df_sorted_product_recommendation


In [ ]:
df_rank = new_df_sorted_product_recommendation[(new_df_sorted_product_recommendation['overall_rating'] != 0) & (new_df_sorted_product_recommendation['overall_rating'] != 1) & (new_df_sorted_product_recommendation['Total_Sum_Aspects'] != 0)] 
df_rank


In [ ]:
tokenizer_1 = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased",)
model_1 = AutoModel.from_pretrained("bert-base-multilingual-uncased",output_hidden_states=True)

In [ ]:
#create embeddings
def get_embeddings(text,token_length):
  tokens = tokenizer_1(text,max_length=token_length,padding='max_length',truncation=False)
  output = model_1(torch.tensor(tokens.input_ids).unsqueeze(0), attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0)).hidden_states[-1]
  return torch.mean(output,axis=1).detach().numpy()

#calculate similarity
def calculate_similarity(text1, text2, token_length=110):
    # Getting the text embeedings
    out1=get_embeddings(text1,token_length=token_length)
    out2=get_embeddings(text2,token_length=token_length)
  
    sim1= cosine_similarity(out1,out2)[0][0]
    print ("Esse é o valor de similaridade utilizando Cosine:", sim1)
    if (sim1 >= 0.80):
        print('Returning True')
        print()
        print()
        return(True)
        
    else:
        print('Returning False')
        print()
        print()
        return(False) 

In [ ]:
def get_BertScore(predictions, references):
  results = bertscore.compute(predictions=predictions, references=references, lang = "portuguese", model_type="bert-base-uncased", num_layers = 8)
  
  predi = ' '.join([str(elem) for elem in predictions])
  refi = ' '.join([str(eleme) for eleme in references])

  if (results['precision'][0] >= 0.65):
      print("Results: Precision maior que 65%", results['precision'][0])
      if ((predi in refi) or (refi in predi)):
         print("Substring True")
         print()
         return(True)
      else:
         print("Substring False")
         print("return false")
         print()
         return(False)
  else:
      print("Similarity menor que 65%")
      return(False)

In [ ]:
df_rank.reset_index(inplace=True)
df_rank

In [ ]:
def product_recommendation(text, similarity_type):
  if (similarity_type == 'BertScore'):
      print("Similarity_Type = BertScore")
      new_df_sorted_product_recommendation = text

      for index, row in new_df_sorted_product_recommendation.iterrows():
  
        if index == 0:
          continue
        else:
          x = 0
          new_df_sorted_product_recommendation.reset_index(drop=True, inplace=True)
          str = (row["product_name"])

          while ((x < index) & (index < len(new_df_sorted_product_recommendation.index))):
            #print(colored(str, 'red', attrs=['bold']))
            
            if (new_df_sorted_product_recommendation['product_name'][x] != "Excluded"):
              print(colored(str, 'red', attrs=['bold']))
              print(colored(new_df_sorted_product_recommendation['product_name'][x], 'blue', attrs=['bold']))

              if ((get_BertScore([str], [new_df_sorted_product_recommendation['product_name'][x]]) ) == True):
                t = new_df_sorted_product_recommendation.loc[index, 'rank']
                z = new_df_sorted_product_recommendation.at[x, 'rank']
                w = t+z
                new_df_sorted_product_recommendation.at[x, 'rank'] = w

                ##Count
                count = new_df_sorted_product_recommendation.loc[x, 'count']
                count_1 = count+1
                new_df_sorted_product_recommendation.at[x, 'count'] = count_1
                new_df_sorted_product_recommendation.at[index,'product_name'] = 'Excluded'
                break
            x = x + 1

  elif(similarity_type == 'Cossine'):
    print("Similarity_Type = Cossine")
    new_df_sorted_product_recommendation = text

    for index, row in new_df_sorted_product_recommendation.iterrows():
  
      if index == 0:
        continue

      else:
        x = 0
      
        new_df_sorted_product_recommendation.reset_index(drop=True, inplace=True)
        str = (row["product_name"])

        while ((x < index) & (index < len(new_df_sorted_product_recommendation.index))):
          print(colored(str, 'red', attrs=['bold']))
          if (new_df_sorted_product_recommendation['product_name'][x] == "Excluded"):
            continue
          print(colored(new_df_sorted_product_recommendation['product_name'][x], 'blue', attrs=['bold']))
          if ((calculate_similarity(str, new_df_sorted_product_recommendation['product_name'][x]) ) == True):
            t = new_df_sorted_product_recommendation.loc[index, 'rank']
            z = new_df_sorted_product_recommendation.at[x, 'rank']
            w = t+z
            new_df_sorted_product_recommendation.at[x, 'rank'] = w

            ##Count
            count = new_df_sorted_product_recommendation.loc[x, 'count']
            count_1 = count+1
            new_df_sorted_product_recommendation.at[x, 'count'] = count_1
           
            new_df_sorted_product_recommendation.at[index,'product_name'] = 'Excluded'
            break
          x = x + 1

  return(new_df_sorted_product_recommendation)

In [ ]:
df_rank_18 = df_rank.head(18)
df_rank_18

In [ ]:
similarity_type = 'BertScore' # BertScore or Cossine
df_ranked_to_the_user = product_recommendation(df_rank, similarity_type)
products_to_the_user = df_ranked_to_the_user[df_ranked_to_the_user["product_name"]!='Excluded']
products_to_the_user


In [ ]:
products_to_the_user = products_to_the_user.sort_values(by=['rank'], ascending=False)
products_to_the_user

,index,product_name,review_title,review_text,Total_Sum_Aspects,rank,overall_rating,count
4,29627,Geladeira / Refrigerador Brastemp Frost Free C...,muito bom,custo e qualidade excelentes.recomendo.Não há ...,2.0,6.157895,5.0,22
2,12171,Geladeira/Refrigerador Consul 2 Portas CRM51 F...,Satisfação garantida,Correspondeu às expectativas. Design moderno e...,3.0,6.105263,4.0,15
0,95354,Geladeira/Refrigerador Consul Frost Free Bem E...,excelente,"produto da melhor qualidade,foi entregue antes...",3.0,4.421053,5.0,14
11,46202,Geladeira / Refrigerador Brastemp Frost Free C...,100% satisfeito,"atendeu todas as expectativas, produto de ótim...",2.0,4.368421,5.0,14
1,104081,Geladeira/ Refrigerador Electrolux Frost Free ...,"Chegou antes do prazo, atendeu bem mais do que...",Realizei a compra do produto em 11/04/2018 as ...,3.0,2.263158,5.0,6
31,93647,Geladeira/Refrigerador Consul Frost Free Duple...,Formidável,Qualidade do produto é tão 10 que dispensa com...,1.0,1.842105,5.0,7
49,109038,Geladeira / Refrigerador Consul Frost Free Dup...,Gostei muito!!,Me surpreendi melhor do que esperava e muito b...,1.0,1.263158,5.0,4
29,99423,Kit 06 Portas Condimento Tempero Inox Magnétic...,Adorei :)),"Recomendo, muito prático, produto de boa quali...",1.0,1.052632,5.0,3
36,93770,Geladeira/Refrigerador Brastemp Frost Free Sid...,"Li as avaliações e fiquei com meno no começo, ...","Pessoal, eu fiz questão de vir aqui compartilh...",1.0,0.947368,5.0,3
37,2766,Geladeira/Refrigerador Electrolux DC51X Cycle ...,Geladeira Eletrolux,Produto excelente. Pesquisei bastante outras m...,1.0,0.736842,5.0,2


In [ ]:
products_to_the_user.to_excel("products_geladeira_to_the_user.xlsx")

In [ ]:
new_df_sorted